In [1]:
%cd ~/Workspace/kaggle-2019Q3-steel/

/home/chengjiun/Workspace/kaggle-2019Q3-steel


In [4]:
import pandas as pd

from mlcomp.contrib.split import stratified_group_k_fold
from mlcomp.worker.executors import Executor

BASE_DIR = '../DATA/kaggle-2019Q3-steel/'

@Executor.register
class Preprocess(Executor):
    def work(self):
        df = pd.read_csv(f'{BASE_DIR}/train.csv')
        df['exists'] = df['EncodedPixels'].notnull().astype(int)

        df['image_name'] = df['ImageId_ClassId'].map(
            lambda x: x.split('_')[0].strip()
        )
        df['class_id'] = df['ImageId_ClassId'].map(
            lambda x: int(x.split('_')[-1])
        )
        df['class_id'] = [
            row.class_id if row.exists else 0 for row in df.itertuples()
        ]
        df['fold'] = stratified_group_k_fold(
            label='class_id', group_column='image_name', df=df, n_splits=5
        )
        df.to_csv(f'TEMP/fold.csv', index=False)


In [5]:
Preprocess().work()

In [8]:
import os

import cv2
import pandas as pd
from tqdm import tqdm
import numpy as np

from mlcomp.contrib.transform.rle import rle2mask
from mlcomp.worker.executors import Executor
from executors.preprocess import BASE_DIR

@Executor.register
class Masks(Executor):
    def work(self) -> dict:
        df = pd.read_csv('TEMP/fold.csv')
        os.makedirs(f'{BASE_DIR}/train_masks', exist_ok=True)

        df = df.sort_values(by='ImageId_ClassId')
        size = (256, 1600)
        mask = np.zeros(size)
        res = []
        for row in tqdm(df.itertuples(), total=df.shape[0]):
            pixels = row.EncodedPixels
            if not isinstance(pixels, str):
                pixels = ''

            mask_cls = rle2mask(pixels, size[::-1])
            mask[mask_cls > 0] = row.class_id

            if row.ImageId_ClassId.endswith('_4'):
                img_id = row.ImageId_ClassId.split('.')[0].strip()
                cv2.imwrite(f'{BASE_DIR}/train_masks/{img_id}.png', mask)
                mask = np.zeros(size)

                res.append(
                    {
                        'fold': row.fold,
                        'image': f'{img_id}.jpg',
                        'mask': f'{img_id}.png'
                    }
                )

        pd.DataFrame(res).to_csv(f'{BASE_DIR}/masks.csv', index=False)

In [9]:
Masks().work()

100%|██████████| 50272/50272 [00:26<00:00, 1862.85it/s]
